**Algorithm 4.2** Import packages, set prime p and precision M.

**Integral ii)**

In [ ]:
"""Paper referenced: https://services.math.duke.edu/~dasgupta/papers/comp.pdf"""
"""TODO: Optimise by adding S where appropriate. Eg: x -> S(x)"""
import sympy,math, gmpy2
# M = 3
# p = 11
# N = 7
S = Qp(p, prec = M*2+1, type = 'capped-rel', print_mode = 'series')	#Defines the padic ring of rationals, with specified precision.
R.<y> = PowerSeriesRing(S)	#Allows us to write polynomials in y with coefficients in S.  

In [ ]:
def BernPolyTilde(s, x):  
    """Same implementation as Algo 4.2a
    Returns: Tilde Bernoulli Polynomial (Pg.6)"""
    if s == 1 and round(x) == x:
        return 0
    else:
        return bernoulli_polynomial(x-math.floor(x),s)
    
def powerSeriesxToTheMinusl(l,M,h):
    """Power series of y^-l expanded around h to precision M.
    DO NOT input h such that p|h. 
    Checked by hand"""
    prec = M + math.ceil(M)
    power_y = 1/((y+h)^l+ O(y^prec))
    power_y = power_y.polynomial()
    return power_y(y-h)

def momentsofF1(h,e,r,l,M,p,N):
    """ Returns the F1 moments as per Proposition 3.1, using the equation on Pg. 7
    """
    total = 0
    #Power series is expressed in decreasing powers.
    powerSeries = powerSeriesxToTheMinusl(l,M,h).polynomial()
    exponents = powerSeries.exponents()
    coeffs = powerSeries.coefficients()
    totalLen = len(exponents)
    
    for i in range(totalLen):
        deg = exponents[i]
        total += coeffs[i]*S(-(e*p^r/N)^deg*BernPolyTilde(deg+1, h*N/(e*p^r))/(deg+1)
                            +N*(e*p^r)^deg*BernPolyTilde(deg+1, h/(e*p^r))/(deg+1))
        
    return total

def integral_ii(momentsofF1_dict,j,n,p,a,N,c,M):
    """j ranges from 0 to p-1, n ranges from 0 to M-1, the basis is [inf] - [a/Nc].
    This algorithm eeds to be changed if we use a different set of basis. 
    Calculate w=b/ep^r. Here we can show that b = a-jN, e = N, r = 1 for our purpose. 
    TODO: Verify what if p divides b? 
    Returns the value at Eqn 37.
    """
    b, e, r = a-j*N, N, 1
    
    if b%p == 0:
        b = b/p
        r = 0
        total = 0
        for l in range(n+1):
            partialSum = 0
            for h in range(1,e+1):
                newMomentSum = 0
                for h_prime in range(1,e*p+1):
                    if h_prime%p != 0 and (h-h_prime)%e == 0: 
                        newMomentSum += momentsofF1_dict[(h_prime,l)]
                partialSum += BernPolyTilde(l+1, h*b/(e*p^r))*newMomentSum/(l+1)
            total -= S(gmpy2.comb(n,l) * (b/(e*p^r))^(n-l) * (-1)^l) * partialSum
    else:
        total = 0
        for l in range(n+1):
            partialSum = 0
            for h in range(1,e*p^r+1):
                if h%p != 0:
                    partialSum += BernPolyTilde(l+1, h*b/(e*p^r))*momentsofF1_dict[(h,l)]/(l+1)
            total -= S(gmpy2.comb(n,l) * (b/(e*p^r))^(n-l) * (-1)^l) * partialSum
            
    return p^n*total*12


def integral_ii_dict(p,N,M):
    """Creates dictionary of integrals, indexed by (i,j,n)
    i denotes the basis [inf]-[i/N], j = 0,...,p-1, n = 0,...,M-1
    """
    momentsofF1_dict = {}
    
    for h in range (1,N*p+1):
        if h%p != 0:
            for l in range(M):
                momentsofF1_dict[(h,l)] = momentsofF1(h,N,1,l,M,p,N)
                
    integral_ii_dict = {}
    
    for i in range(1,math.floor(N/2)+1):
        for j in range(p):
            for n in range(M):
                integral_ii_dict[(i,j,n)] =  integral_ii(momentsofF1_dict,j,n,p,i,N,1,M) + O(p^(M+1))
                
    return integral_ii_dict

**Integral iv)**

In [22]:
def powerSeriesEqn43(n,N,j,M):
    """Expanding (u/-Nu+1)^-n around j=1/N(mod p) as per Eqn. 43. 
    Returns a power series in terms of y = (u-j). """
    prec = M + math.ceil(M)
    poly = ((1-j*N-N*y)/(y+j+ O(y^prec)))^n 
    poly = poly.polynomial()
    return poly

def integral_iv_auxilliary(momentsofF1_dict,n,p,a,N,c,M):
    """Calculates using a similar method as per integral ii). 
    Returns integral of (u-j)^n, used for calculating power series in Eqn 43."""
    j = N.inverse_mod(p)
    #Basis is [inf] - [a/Nc]. We write gamma^-1*([inf] - [a/Nc]) = [inf]-[1/-N] + [inf]-[a/N(c-a)] 
    total = 0
    #First do [inf]-[1/-N]
    b, e, r = -1-j*N, N, 1
    for l in range(n+1):
        partialSum = 0
        for h in range(1,e*p^r+1):
            if h%p != 0:
                partialSum += BernPolyTilde(l+1, h*b/(N*p))*momentsofF1_dict[(h,l)]/(l+1)
        total += S(gmpy2.comb(n,l) * (b/(e*p^r))^(n-l) * (-1)^l) * partialSum
    #Now do [inf]-[a/N(c-a)]
    if a == c:
        pass
    else:
        b, e, r = -a+j*N*(c-a), N*(a-c), 1
        if b%p == 0:
            b = b/p
            r = 0
            for l in range(n+1):
                partialSum = 0
                for h in range(1,e+1):
                    newMomentSum = 0
                    for h_prime in range(1,e*p+1):
                        if h_prime%p != 0 and (h-h_prime)%e == 0: 
                            newMomentSum += momentsofF1_dict[(h_prime,l)]
                    partialSum += BernPolyTilde(l+1, h*b/(e*p^r))*newMomentSum/(l+1)
                total -= S(gmpy2.comb(n,l) * (b/(e*p^r))^(n-l) * (-1)^l) * partialSum
        else:
            for l in range(n+1):
                partialSum = 0
                for h in range(1,e*p^r+1):
                    if h%p != 0:
                        partialSum += BernPolyTilde(l+1, h*b/(e*p^r))*momentsofF1_dict[(h,l)]/(l+1)
                total -= S(gmpy2.comb(n,l) * (b/(e*p^r))^(n-l) * (-1)^l) * partialSum
                
    return p^n*total*12

def integral_iv(integral_iv_aux_dict,n,p,a,N,c,M):
    """Returns integral_iv, using the power series expansion of Eqn 43, 
    and each term (u-j)^k is given in integral_iv_aux_dict."""
    j = N.inverse_mod(p)
    #Power series is expressed in decreasing powers.
    powerSeries = powerSeriesEqn43(n,N,j,M)
    exponents = powerSeries.exponents()
    coeffs = powerSeries.coefficients()
    totalLen = len(exponents)
    
    total = 0
    for i in range(totalLen):
        deg = exponents[i]
        total += coeffs[i] * integral_iv_aux_dict[(a,deg)]
    
    return total
    
def integral_iv_dict(p,N,M):
    """Creates dictionary of integrals, indexed by (i,n)
    i denotes the basis [inf]-[i/N], n = 0,...,M-1
    """
    momentsofF1_dict = {}
    for h in range (1,N*p*(math.floor(N/2)-1)+1):
        if h%p != 0:
            for l in range(M*2):
                momentsofF1_dict[(h,l)] = momentsofF1(h,N*p,0,l,M,p,N)
                
    integral_iv_aux_dict = {} 
    for i in range(1,math.floor(N/2)+1):
        for n in range(M*2):
            integral_iv_aux_dict[(i,n)] = integral_iv_auxilliary(momentsofF1_dict,n,p,i,N,1,M)
    
    integral_iv_dict = {}
    for i in range(1,math.floor(N/2)+1):
        for n in range(M):
            integral_iv_dict[(i,n)] = integral_iv(integral_iv_aux_dict,n,p,i,N,1,M) + O(p^(M+1))
            
    return integral_iv_dict

In [23]:
if __name__ == '__main__' and '__file__' not in globals():
    print(integral_ii_dict(p,N,M))
    print(integral_iv_dict(p,N,M))

{(1, 0, 0): 4 + 7*11^2 + 10*11^3 + O(11^4), (1, 0, 1): 3*11^3 + O(11^4), (1, 0, 2): 7*11^3 + O(11^4), (1, 1, 0): 10 + 10*11 + O(11^4), (1, 1, 1): 10*11 + 3*11^2 + 3*11^3 + O(11^4), (1, 1, 2): 5*11^2 + O(11^4), (1, 2, 0): 10 + 10*11 + O(11^4), (1, 2, 1): 6*11 + 11^2 + 3*11^3 + O(11^4), (1, 2, 2): 5*11^2 + 9*11^3 + O(11^4), (1, 3, 0): 6 + 6*11 + O(11^4), (1, 3, 1): 8*11 + 11^2 + 6*11^3 + O(11^4), (1, 3, 2): 10*11^2 + 4*11^3 + O(11^4), (1, 4, 0): 6 + 6*11 + O(11^4), (1, 4, 1): 8*11 + 8*11^2 + 10*11^3 + O(11^4), (1, 4, 2): 11^2 + 7*11^3 + O(11^4), (1, 5, 0): O(11^4), (1, 5, 1): 7*11^2 + 10*11^3 + O(11^4), (1, 5, 2): 2*11^2 + O(11^4), (1, 6, 0): 4 + 4*11 + O(11^4), (1, 6, 1): 4*11 + 7*11^2 + O(11^4), (1, 6, 2): 4*11^2 + 5*11^3 + O(11^4), (1, 7, 0): 9 + 8*11 + 10*11^2 + 10*11^3 + O(11^4), (1, 7, 1): 10*11 + 11^2 + 6*11^3 + O(11^4), (1, 7, 2): 9*11^2 + O(11^4), (1, 8, 0): 2 + 2*11 + O(11^4), (1, 8, 1): 6*11 + 4*11^2 + 9*11^3 + O(11^4), (1, 8, 2): 2*11^3 + O(11^4), (1, 9, 0): 4 + 4*11 + O(11^4